In [40]:
import pandas as pd

In [41]:
sentiment = pd.read_csv('Dataset/imdb_labelled.txt', sep='\t', names = ['comment','label'])


In [42]:
sentiment.head()

,comment,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [43]:
sentiment.shape

(748, 3)

In [44]:
sentiment.describe()

,label,length
count,748.000000,748.000000
mean,0.516043,111.004011
std,0.500077,380.660191
min,0.000000,7.000000
25%,0.000000,42.750000
50%,1.000000,72.000000
75%,1.000000,112.250000
max,1.000000,7944.000000


In [45]:
import numpy as np
np.sum((sentiment['label']))

386

In [46]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 3 columns):
comment    748 non-null object
label      748 non-null int64
length     748 non-null int64
dtypes: int64(2), object(1)
memory usage: 17.6+ KB


In [47]:
sentiment.groupby('label').describe()

length                                                          
       count        mean         std  min    25%   50%     75%     max
label                                                                 
0      362.0  117.185083  489.595391  8.0  42.00  68.0  104.00  7944.0
1      386.0  105.207254  237.312394  7.0  43.25  77.0  123.75  4488.0

In [48]:
#verify length of messages and also add it as a column
sentiment['length']= sentiment['comment'].apply(len)

In [49]:
sentiment.head()

,comment,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [50]:
sentiment[sentiment['length']>50]['comment'].iloc[0]

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '

In [51]:
#start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [52]:
#define function to clean stopwords
def cleanup_text(msg):
    no_punctuation = [char for char in msg if char not in string.punctuation]
    no_punctuation_msg = ''.join(no_punctuation)
    return [word for word in no_punctuation_msg.split() if word.lower() not in stopwords.words('english') ]

In [53]:
import string
from nltk.corpus import stopwords
bagOfWords =  CountVectorizer(analyzer = cleanup_text).fit(sentiment['comment'])

In [54]:
bagOfWords

CountVectorizer(analyzer=<function cleanup_text at 0x11684b620>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [55]:
#apply transform method for bag of owrds
comment_bagOfWords = bagOfWords.transform(sentiment['comment'])

In [56]:
print(comment_bagOfWords[0])

  (0, 808)	1
  (0, 1350)	1
  (0, 1377)	1
  (0, 2066)	1
  (0, 2153)	1
  (0, 2723)	1
  (0, 3250)	1


In [27]:
len(bagOfWords.vocabulary_)

3259

In [28]:
#Apply tfidf transformer and fit the bag of words into it
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(comment_bagOfWords)

In [30]:
comment_tfidf = tfidf_transformer.transform(comment_bagOfWords)
print(comment_tfidf.shape)

(748, 3259)


In [39]:
type(comment_tfidf)

scipy.sparse.csr.csr_matrix

In [57]:
from sklearn.naive_bayes import MultinomialNB
sentiment_model = MultinomialNB().fit(comment_tfidf, sentiment['label'])

In [37]:
#check model
comment = sentiment['comment'][241]
print(comment)
bag_of_words_for_comment = bagOfWords.transform([comment])
tfidf_inst = tfidf_transformer.transform(bag_of_words_for_comment)
print('Predicted class: ', sentiment_model.predict(tfidf_inst)[0])
print('Expected class:  ', sentiment.label[241])

Yes, I am simplifying things here for the sake of brevity, for this really is at the core of the problems with this film - it has too much going on without any real, fulfilling explanation.  
Predicted class:  0
Expected class:   0
